In [33]:
import numpy as np
import pandas as pd

from ols_bootstrap.pairs import PairsBootstrap
from ols_bootstrap.residual import ResidualBootstrap
from ols_bootstrap.wild import WildBootstrap

In [34]:
df = pd.read_csv('./balance2018.csv')
df = df[df['sales_clean'] != 0]
df = df[['sales_clean', 'tanass_clean', 'tax']]
df = df.dropna(subset=['tanass_clean', 'tax'])

df_scaled = df.applymap(lambda x: np.log(x + 1))

/home/phamv/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (66) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [35]:
df_sample = df_scaled.sample(n=10000, replace=False)

Y_data = pd.DataFrame(df_sample.iloc[:, 0])
X_data = pd.DataFrame(df_sample.iloc[:, 1:])

In [36]:
X_data.head()

,tanass_clean,tax
395710,4.948760,0.693147
192436,9.089415,4.204693
285587,13.419153,7.754053
155181,11.881913,7.105786
162194,5.752573,4.955827


In [37]:
Y_data.head()

,sales_clean
395710,7.739794
192436,11.036582
285587,11.323350
155181,12.557089
162194,7.346010


In [38]:
psb = PairsBootstrap(Y_data, X_data, iter = 100)
rsb = ResidualBootstrap(Y_data, X_data, iter = 100)
wb_stdn = WildBootstrap(Y_data, X_data, iter = 100, from_distro = 'standard_normal')
wb_rad = WildBootstrap(Y_data, X_data, iter = 100, from_distro = 'rademacher')
wb_mam = WildBootstrap(Y_data, X_data, iter = 100, from_distro = 'mammen')

In [39]:
psb.summary()

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                            Pairs Bootstrap results with sample size of 10000 and bootstrap resampling size of 100 using 95.00% CI                                            |
+----------------+------------------+-------------------------------+-----------------+-----------------------------+-------------------+----------+----------+--------------------+-----------+
|     Params     |  Original Coeff  |  Mean of Bootstrapped Coeffs  |  Orig Coeff SE  |  SE of Bootstrapped Coeffs  |  % of Diff in SE  |   Bias   |   RMSE   |         CI         |  CI Diff  |
+----------------+------------------+-------------------------------+-----------------+-----------------------------+-------------------+----------+----------+--------------------+-----------+
|     const      |      6.2094     

In [40]:
rsb.summary()

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                          Residual Bootstrap results with sample size of 10000 and bootstrap resampling size of 100 using 95.00% CI                                           |
+----------------+------------------+-------------------------------+-----------------+-----------------------------+-------------------+----------+----------+--------------------+-----------+
|     Params     |  Original Coeff  |  Mean of Bootstrapped Coeffs  |  Orig Coeff SE  |  SE of Bootstrapped Coeffs  |  % of Diff in SE  |   Bias   |   RMSE   |         CI         |  CI Diff  |
+----------------+------------------+-------------------------------+-----------------+-----------------------------+-------------------+----------+----------+--------------------+-----------+
|     const      |      6.2094     

In [41]:
wb_stdn.summary()

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                  Wild Bootstrap with Standard Normal results with sample size of 10000 and bootstrap resampling size of 100 using 95.00% CI                                  |
+----------------+------------------+-------------------------------+-----------------+-----------------------------+-------------------+----------+----------+--------------------+-----------+
|     Params     |  Original Coeff  |  Mean of Bootstrapped Coeffs  |  Orig Coeff SE  |  SE of Bootstrapped Coeffs  |  % of Diff in SE  |   Bias   |   RMSE   |         CI         |  CI Diff  |
+----------------+------------------+-------------------------------+-----------------+-----------------------------+-------------------+----------+----------+--------------------+-----------+
|     const      |      6.2094     

In [42]:
wb_rad.summary()

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                    Wild Bootstrap with Rademacher results with sample size of 10000 and bootstrap resampling size of 100 using 95.00% CI                                     |
+----------------+------------------+-------------------------------+-----------------+-----------------------------+-------------------+----------+----------+--------------------+-----------+
|     Params     |  Original Coeff  |  Mean of Bootstrapped Coeffs  |  Orig Coeff SE  |  SE of Bootstrapped Coeffs  |  % of Diff in SE  |   Bias   |   RMSE   |         CI         |  CI Diff  |
+----------------+------------------+-------------------------------+-----------------+-----------------------------+-------------------+----------+----------+--------------------+-----------+
|     const      |      6.2094     

In [44]:
wb_mam.summary()

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                      Wild Bootstrap with Mammen results with sample size of 10000 and bootstrap resampling size of 100 using 95.00% CI                                       |
+----------------+------------------+-------------------------------+-----------------+-----------------------------+-------------------+----------+----------+--------------------+-----------+
|     Params     |  Original Coeff  |  Mean of Bootstrapped Coeffs  |  Orig Coeff SE  |  SE of Bootstrapped Coeffs  |  % of Diff in SE  |   Bias   |   RMSE   |         CI         |  CI Diff  |
+----------------+------------------+-------------------------------+-----------------+-----------------------------+-------------------+----------+----------+--------------------+-----------+
|     const      |      6.2094     